In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:
cd drive/MyDrive/IE7500_GroupB/Notebooks

/content/drive/.shortcut-targets-by-id/1hXOWgLy2A2GiwQDqG1SudIATn6Z_m1b6/IE7500_GroupB/Notebooks


In [1]:
# load necesary libraries
import numpy as np
import pandas as pd

In [3]:
dtypes_dict = {'headline': 'object',
               'url': 'object',
               'publisher': 'object',
               'stock': 'object',
               'tokens': 'object',
               'normalized_tokens': 'object',
               'filtered_tokens': 'object',
               'lemmas': 'object',
               'sentiment_score': 'float64',
               'Name': 'object',
               'Market Cap': 'float64',
               'Country': 'object',
               'IPO Year': 'float64',
               'Sector': 'object',
               'Industry': 'object',
               'year': 'int32',
               'month': 'int32',
               'day_of_week': 'int32',
               'sentiment_label': 'int64',
               'headline_length': 'int64',
               'word_count': 'int64',
               'Market_Cap_Category': 'object',
               'recommendation': 'object',
               'cap_Large': 'bool',
               'cap_Medium': 'bool',
               'cap_Mega': 'bool',
               'cap_Micro': 'bool',
               'cap_Nano': 'bool',
               'cap_Small': 'bool',
               'sector_Basic Materials': 'bool',
               'sector_Consumer Discretionary': 'bool',
               'sector_Consumer Staples': 'bool',
               'sector_Energy': 'bool',
               'sector_Finance': 'bool',
               'sector_Health Care': 'bool',
               'sector_Industrials': 'bool',
               'sector_Miscellaneous': 'bool',
               'sector_Real Estate': 'bool',
               'sector_Technology': 'bool',
               'sector_Telecommunications': 'bool',
               'sector_Utilities': 'bool', 'recommendation_label': 'int64',
               'publisher_label': 'int64', 'country_label': 'int64',
               'industry_label': 'int64'}

In [8]:
# load dataframes to use
df_main = pd.read_csv("saved_dfs/df_for_models.csv", dtype=dtypes_dict,
                      parse_dates=['date'])

In [ ]:
df_main.head()

,headline,url,publisher,date,stock,tokens,normalized_tokens,filtered_tokens,lemmas,sentiment_score,...,sector_Industrials,sector_Miscellaneous,sector_Real Estate,sector_Technology,sector_Telecommunications,sector_Utilities,recommendation_label,publisher_label,country_label,industry_label
0,Agilent Technologies Announces Pricing of $5……...,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01,A,"['Agilent', 'Technologies', 'Announces', 'Pric...","['agilent', 'technologies', 'announces', 'pric...","['agilent', 'technologies', 'announces', 'pric...","['agilent', 'technology', 'announces', 'pricin...",0.0000,...,True,False,False,False,False,False,1,4,45,18
1,Agilent (A) Gears Up for Q2 Earnings: What's i...,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18,A,"['Agilent', '(', 'A', ')', 'Gears', 'Up', 'for...","['agilent', 'a', 'gears', 'up', 'for', 'q2', '...","['agilent', 'gears', 'q2', 'earnings', 'cards']","['agilent', 'gear', 'q2', 'earnings', 'card']",0.0000,...,True,False,False,False,False,False,1,16,45,18
2,J.P. Morgan Asset Management Announces Liquida...,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15,A,"['J.P.', 'Morgan', 'Asset', 'Management', 'Ann...","['morgan', 'asset', 'management', 'announces',...","['morgan', 'asset', 'management', 'announces',...","['morgan', 'asset', 'management', 'announces',...",0.3612,...,True,False,False,False,False,False,1,4,45,18
3,"Pershing Square Capital Management, L.P. Buys ...",http://www.gurufocus.com/news/1138704/pershing...,GuruFocus,2020-05-15,A,"['Pershing', 'Square', 'Capital', 'Management'...","['pershing', 'square', 'capital', 'management'...","['pershing', 'square', 'capital', 'management'...","['pershing', 'square', 'capital', 'management'...",0.0000,...,True,False,False,False,False,False,1,4,45,18
4,Agilent Awards Trilogy Sciences with a Golden ...,http://www.gurufocus.com/news/1134012/agilent-...,GuruFocus,2020-05-12,A,"['Agilent', 'Awards', 'Trilogy', 'Sciences', '...","['agilent', 'awards', 'trilogy', 'sciences', '...","['agilent', 'awards', 'trilogy', 'sciences', '...","['agilent', 'award', 'trilogy', 'science', 'go...",0.4588,...,True,False,False,False,False,False,1,4,45,18


## Load and Prepare Data

In [9]:
# Define Features
stock_features = ["stock", "industry_label"]  # Stock-related
sector_features = ["sector_Industrials", "sector_Miscellaneous", "sector_Real Estate",
                   "sector_Technology", "sector_Telecommunications", "sector_Utilities"]  # Sector indicators
sentiment_features = ["sentiment_score"]  # Aggregated sentiment
publisher_country = ["publisher_label", "country_label"]
text_features = ["headline", "lemmas"]

# Create DataFrame for Recommendation
recommendation_df = df_main[stock_features + sector_features + sentiment_features + publisher_country + text_features]

# Aggregate sentiment scores per stock (if multiple entries exist)
recommendation_df = recommendation_df.groupby("stock").agg({
    "industry_label": "first",
    **{col: "first" for col in sector_features},  # Keep first occurrence
    "sentiment_score": "mean",  # Aggregate sentiment
    "publisher_label": "first",
    "country_label": "first",
    "headline": "last",  # Most recent headline
    "lemmas": "last"  # Most recent processed text
}).reset_index()

print(recommendation_df.head())

  stock  industry_label  sector_Industrials  sector_Miscellaneous  \
0     A              18                True                 False   
1    AA               6                True                 False   
2   AAC             143               False                 False   
3  AADR             143               False                 False   
4   AAL               5               False                 False   

   sector_Real Estate  sector_Technology  sector_Telecommunications  \
0               False              False                      False   
1               False              False                      False   
2               False              False                      False   
3               False              False                      False   
4               False              False                      False   

   sector_Utilities  sentiment_score  publisher_label  country_label  \
0             False         0.067877                4             45   
1             

# Method 1: Content-Based Filtering (TF-IDF + Cosine Similarity)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Convert 'lemmas' into TF-IDF vectors (text feature extraction)
tfidf_vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2), max_df=0.9, min_df=2)
X_tfidf = tfidf_vectorizer.fit_transform(recommendation_df["lemmas"])


In [11]:
# Compute Cosine Similarity Matrix
cosine_sim = cosine_similarity(X_tfidf, X_tfidf)

# Function to recommend similar stocks based on text similarity
def recommend_stocks_content(stock_name, top_n=5):
    stock_idx = recommendation_df.index[recommendation_df["stock"] == stock_name][0]  # Get stock index
    sim_scores = list(enumerate(cosine_sim[stock_idx]))  # Retrieve similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]  # Sort & exclude self
    stock_indices = [i[0] for i in sim_scores]
    return recommendation_df.iloc[stock_indices][["stock", "industry_label", "headline"]]  # Return recommended stocks

# Example: Recommend stocks similar to stock "A"
print(recommend_stocks_content("A"))


     stock  industry_label                                           headline
790   BSJK             143                         BBBs: Beyond The Headlines
3243  JBHT             138                     JPMorgan: Tomorrow's Headlines
1986    EV             143                  CPI Report: Wednesday's Headlines
3395  KEYS              65  Danaher Upgraded As Analyst Pushes Agilent Buyout
3055  IMPV             143  Security Firm Imperva Follows Groupon With Hot...


# Method 2: KNN Similarity-Based Recommendation

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Select numerical features for similarity search
knn_features = sector_features + ["sentiment_score", "publisher_label", "country_label"]

# Scale the features to ensure uniformity
scaler = StandardScaler()
X_knn = scaler.fit_transform(recommendation_df[knn_features])

# Train the KNN model for similarity search
knn_model = NearestNeighbors(n_neighbors=5, metric="cosine")
knn_model.fit(X_knn)


NearestNeighbors(metric='cosine')

In [13]:
# Function to find similar stocks using KNN
def recommend_stocks_knn(stock_name, top_n=5):
    stock_idx = recommendation_df.index[recommendation_df["stock"] == stock_name][0]  # Find stock index
    distances, indices = knn_model.kneighbors([X_knn[stock_idx]], n_neighbors=top_n+1)  # Retrieve nearest stocks
    stock_indices = indices[0][1:]  # Exclude the stock itself
    return recommendation_df.iloc[stock_indices][["stock", "industry_label", "sentiment_score"]]

# Example: Recommend stocks similar to "A"
print(recommend_stocks_knn("A"))


     stock  industry_label  sentiment_score
3243  JBHT             138         0.067907
394    ASA             109         0.067978
2884    HY              37         0.067775
990     CF               4         0.067628
2490   GHM              65         0.068211


# Method 3: Clustering-Based Stock Recommendation (K-Means)

In [14]:
from sklearn.cluster import KMeans

# Train K-Means model to create stock groups
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
recommendation_df["cluster"] = kmeans.fit_predict(X_knn)

# Print stocks with their assigned clusters
print(recommendation_df[["stock", "industry_label", "cluster"]].head(10))


  stock  industry_label  cluster
0     A              18        0
1    AA               6        0
2   AAC             143        2
3  AADR             143        2
4   AAL               5        2
5  AAMC             143        2
6  AAME              70        2
7   AAN             143        2
8  AAOI             127        3
9  AAON              65        0


In [15]:
# Function to recommend stocks from the same cluster
def recommend_stocks_cluster(stock_name, top_n=5):
    cluster_label = recommendation_df.loc[recommendation_df["stock"] == stock_name, "cluster"].values[0]
    cluster_stocks = recommendation_df[recommendation_df["cluster"] == cluster_label]  # Get stocks in the same cluster
    return cluster_stocks.sample(min(top_n, len(cluster_stocks)))

# Example: Recommend stocks in the same cluster as "A"
print(recommend_stocks_cluster("A"))

     stock  industry_label  sector_Industrials  sector_Miscellaneous  \
2208   FIX              50                True                 False   
5817   TNC              65                True                 False   
1       AA               6                True                 False   
5866  TRMB              65                True                 False   
2737  HEES              83                True                 False   

      sector_Real Estate  sector_Technology  sector_Telecommunications  \
2208               False              False                      False   
5817               False              False                      False   
1                  False              False                      False   
5866               False              False                      False   
2737               False              False                      False   

      sector_Utilities  sentiment_score  publisher_label  country_label  \
2208             False         0.250866        

# Method 4: Collaborative Filtering (Deep Learning)

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate

# Define Inputs (Investor & Stock Embeddings)
input_investor = Input(shape=(1,))
input_stock = Input(shape=(1,))

# Define Embeddings
embedding_investor = Embedding(input_dim=100, output_dim=10)(input_investor)
embedding_stock = Embedding(input_dim=1000, output_dim=10)(input_stock)

# Flatten Layers
vector_investor = Flatten()(embedding_investor)
vector_stock = Flatten()(embedding_stock)

# Concatenate & Predict Interaction Score
concat = Concatenate()([vector_investor, vector_stock])
dense = Dense(128, activation="relu")(concat)
output = Dense(1, activation="sigmoid")(dense)

# Compile Model
model = Model(inputs=[input_investor, input_stock], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 1, 10)          │          1,000 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 1, 10)          │         10,000 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 10)             │              0 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_1 (Flatten)       │ (None, 10)             │              0 │ embedding_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 20)             │              0 │ flatten[0][0],         │
│                           │                        │                │ flatten_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │          2,688 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1)              │            129 │ dense[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 13,817 (53.97 KB)

 Trainable params: 13,817 (53.97 KB)

 Non-trainable params: 0 (0.00 B)